In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!jupyter nbconvert RunExperiment_Sample_On_Snowpark.ipynb --to python --output run_experiment.py

In [ ]:
from run_experiment import run_exp

In [ ]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf
import pandas as pd

In [ ]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
    }
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

In [ ]:
session.add_import("run_experiment.py", import_path=".run_experiment")
session.add_packages("numpy", "pandas", "snowflake", "snowflake-ml-python")

In [ ]:
# def udf_run_exp(sf_pass: str, dataset: str, target: str) -> pd.DataFrame:
#     return run_exp(sf_pass, dataset, target)
from snowflake.snowpark.types import StructType, StructField, IntegerType, StringType, ArrayType

In [ ]:
udf_run_exp = udf(
    func=run_exp,
    return_type=StringType(),
    input_types=[ArrayType(StringType()),StringType(),StringType()],
    name="udf_run_exp",
    session=session
    )

In [ ]:
session

In [ ]:
password = os.environ.get('SF_Password')

In [ ]:
out = session.sql(f"select udf_run_exp(ARRAY_CONSTRUCT('snowflake.ml.modeling.naive_bayes.GaussianNB'),'EMPLOYEE', 'LEAVEORNOT') as run").collect()

In [ ]:
@udf
def add_udf(x: int, y: int) -> int:
    return x + y
df = session.create_dataframe([[1, 2], [3, 4]], schema=["x", "y"])
df.select(add_udf("x", "y")).to_df("add_result").collect()

In [ ]:
algos=['snowflake.ml.modeling.naive_bayes.GaussianNB',
       'snowflake.ml.modeling.neighbors.KNeighborsClassifier',
      ]

In [ ]:
session.close()